# CPX / CTRU Data checks
### Last update June 2, 2024
### Aubrey Roberts

In [333]:
# Load packages
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import statsmodels.api as sm
import csv
import re

## Single use case: how to extract VO2 peak

In [353]:
# Specify data of interest 
path = "/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/cpet_bxb/shc_vo2-max/baseline/30_scr_bl_ep_arm_1_bl_cpx_raw_data.xlsx"

# Read in first CPX sheet (name, test date)
df = pd.read_excel(path)
df1 = pd.DataFrame(df)

test_date=df1.columns[4]
first_name = df1.iat[1,1]
last_name = df1.iat[0,1]

# Extract record_id from path name
string = str.split(path,'/')[11]
record_id = str.split(string,'_')[0]
timepoint = str.split(path,'/')[10]

# Print participant
#record_id, first_name, last_name, timepoint, test_date

# Read the second sheet (for VO2 peak)
df2 = pd.read_excel(path, sheet_name='Results', skiprows=13)
df2 = df2[df2['Parameter'] == 'VO2/Kg'][['Max']]

df2['date'] = test_date
df2['record_id'] = record_id
df2['last_name'] = last_name
df2['timepoint'] = timepoint
df2.reset_index(drop=True, inplace=True)
col = ["record_id"]
df2 = df2[col + [x for x in df2.columns if x not in col]]
df2

,record_id,Max,date,last_name,timepoint
0,30,44.4,8/31/2023,Trotsyuk,baseline


## Define function for getting VO2 peak from CPX file

In [335]:
def process_file(path):
    # Read in first CPX sheet (name, test date)
    df = pd.read_excel(path)
    df1 = pd.DataFrame(df)

    test_date = df1.columns[4]
    first_name = df1.iat[1,1]
    last_name = df1.iat[0,1]

    # Extract record_id and timepoint from path name
    string = str.split(path, '/')[11]
    record_id = str.split(string, '_')[0]
    timepoint = str.split(path, '/')[10]

    # Read the second sheet (for VO2 peak)
    df2 = pd.read_excel(path, sheet_name='Results', skiprows=13)
    df2 = df2[df2['Parameter'] == 'VO2/Kg'][['Max']]

    df2['date'] = test_date
    df2['record_id'] = record_id
    df2['last_name'] = last_name
    df2['timepoint'] = timepoint
    df2.reset_index(drop=True, inplace=True)
    col = ["record_id"]
    df2 = df2[col + [x for x in df2.columns if x not in col]]
    df2
    
    return df2

process_file(path)

,record_id,Max,date,last_name,timepoint
0,30,44.4,8/31/2023,Trotsyuk,baseline


### Get all files in Google Drive folder

In [336]:
# Define the path to your directory
import glob
import os
directory_path = "/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/cpet_bxb/shc_vo2-max/baseline/"

# Use glob to get a list of all Excel files in the directory
file_paths = glob.glob(os.path.join(directory_path, "*.xlsx"))

### Get VO2 peaks

In [337]:
# Run this function

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

# Loop over the list of pathnames and process each file
for pathname in file_paths:
    try:
        result = process_file(pathname)
        results_df = pd.concat([results_df, result], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {pathname}: {e}")

# Print or save the consolidated results
print(results_df)
# Optionally, save to a file
# results_df.to_excel('consolidated_results.xlsx', index=False)

   record_id   Max       date     last_name timepoint
0         84  37.5   2/9/2024       Kononov  baseline
1          7  35.9  4/14/2023        ZUSHIN  baseline
2         94    37   3/6/2024     Mostafavi  baseline
3         83  24.9  1/30/2024       Heather  baseline
4          6  30.5  4/13/2023    SASINOWSKI  baseline
..       ...   ...        ...           ...       ...
63        91  37.2  3/22/2024  Jayachandran  baseline
64        86  37.4   3/8/2024       Markley  baseline
65        39  38.5   8/1/2023        BARLET  baseline
66         5  24.9  5/19/2023            SU  baseline
67        19  32.5  6/21/2023         MATTY  baseline

[68 rows x 5 columns]


In [338]:
results_df['record_id']=pd.to_numeric(results_df["record_id"])


## Repeat for Endpoint Data

In [339]:
directory_path = "/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/cpet_bxb/shc_vo2-max/endpoint/"

# Use glob to get a list of all Excel files in the directory
file_paths = glob.glob(os.path.join(directory_path, "*.xlsx"))

# Initialize an empty DataFrame to store results
results_df2 = pd.DataFrame()

# Loop over the list of pathnames and process each file
for pathname in file_paths:
    try:
        result = process_file(pathname)
        results_df2 = pd.concat([results_df2, result], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {pathname}: {e}")

# Print or save the consolidated results
results_df2['record_id']=pd.to_numeric(results_df2["record_id"])
results_df2 = pd.DataFrame(results_df2)
results_df = pd.DataFrame(results_df)

# Combine BL and EP dfs
vo2_df = pd.concat([results_df, results_df2])

#vo2_df

In [340]:
pd.DataFrame(vo2_df)
vo2_df['record_id']=pd.to_numeric(vo2_df['record_id'])
vo2_df = vo2_df.sort_values(by=['record_id'])

In [341]:
vo2_df

,record_id,Max,date,last_name,timepoint
50,3,33,6/20/2023,DELWEL,baseline
1,3,35.2,10/17/2023,DELWEL,endpoint
4,4,28.5,8/3/2023,LEE,endpoint
55,4,34.7,4/21/2023,LEE,baseline
66,5,24.9,5/19/2023,SU,baseline
...,...,...,...,...,...
43,105,38.2,4/30/2024,Duke,baseline
59,109,29.8,4/3/2024,Nelson,baseline
17,110,24.1,5/14/2024,Sahai,baseline
44,112,37.4,5/13/2024,Polett,baseline


## Write to Desktop

In [342]:
path = '/Users/aubreykr/Desktop/'
vo2_df.to_csv(path + 'test.csv', index = False)

## Code for troubleshooting

In [308]:
path = '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/cpet_bxb/shc_vo2-max/baseline/30_scr_bl_ep_arm_1_ep_cpx_raw_data.xlsx'
df = pd.read_excel(path)
df1 = pd.DataFrame(df)
df1

test_date = df1.columns[4]
first_name = df1.iat[1,1]
last_name = df1.iat[0,1]

# Extract record_id and timepoint from path name
string = str.split(path, '/')[11]
record_id = str.split(string, '_')[0]
timepoint = str.split(path, '/')[10]

timepoint, record_id, string

# Read the second sheet (for VO2 peak)
df2 = pd.read_excel(path, sheet_name='Results', skiprows=13)
df2 = df2[df2['Parameter'] == 'VO2/Kg'][['Max']]
df2['date'] = test_date
df2['record_id'] = record_id
df2['last_name'] = last_name
df2['timepoint'] = timepoint
df2.reset_index(drop=True, inplace=True)
col = ["record_id"]
df2 = df2[col + [x for x in df2.columns if x not in col]]
df2

vo2_df[vo2_df['record_id']==26]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aubreykr/Google Drive/Shared drives/HIIT and Endurance Study/Data/data/cpet_bxb/shc_vo2-max/baseline/30_scr_bl_ep_arm_1_ep_cpx_raw_data.xlsx'

## Compare with REDCap data

In [ ]:
# Import REDCap data for comparison